In [1]:
kernel.silent(true)

In [2]:
import coursierapi.MavenRepository
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

In [3]:
import $ivy.`com.github.propi:rdfrules:1.5.0`
import collection._
import org.apache.jena.riot.Lang

import com.github.propi.rdfrules.data._
import com.github.propi.rdfrules.algorithm.amie._
import com.github.propi.rdfrules.algorithm.dbscan._
import com.github.propi.rdfrules.utils._
import com.github.propi.rdfrules.index._
import com.github.propi.rdfrules.rule._
import com.github.propi.rdfrules.ruleset._
kernel.silent(false)

In [4]:
val salariesDataset = Dataset.fromCache("../cache/salariesCube.cache")
salariesDataset.size

salariesDataset: Dataset = com.github.propi.rdfrules.data.Dataset@6b5d8f6a
res3_1: Int = 2898

In [5]:
val refPeriodLinking = Dataset("../data/linking/refPeriodLinking.ttl")
refPeriodLinking.size

refPeriodLinking: Dataset = com.github.propi.rdfrules.data.Dataset@755e7b58
res4_1: Int = 84

In [6]:
val wikidata = Dataset.fromCache("../cache/wikidata.cache")
wikidata.size

wikidata: Dataset = com.github.propi.rdfrules.data.Dataset@37033923
res5_1: Int = 24410

In [7]:
val czsoWikidataLinking = Dataset("../data/linking/czsoWikidataLinking.ttl")
czsoWikidataLinking.size

czsoWikidataLinking: Dataset = com.github.propi.rdfrules.data.Dataset@81b27a6
res6_1: Int = 90

In [8]:
val dataset = salariesDataset + refPeriodLinking + wikidata + czsoWikidataLinking
dataset.size

dataset: Dataset = com.github.propi.rdfrules.data.Dataset@673ea90a
res7_1: Int = 27482

In [9]:
val index = dataset.index()

index: Index = com.github.propi.rdfrules.index.Index$FromDatasetFullyPreservedIndex@4729c892

In [10]:
val uri = (value: String) => TripleItem.Uri(value)
val appliesTo = uri("http://kizi.vse.cz/novp19/diploma-thesis/appliesToRefPeriod")
val wdProperty = (value: Int) => uri("http://www.wikidata.org/prop/P" + value)
val qbDataSet = uri("http://purl.org/linked-data/cube#dataSet")

// salariesMeasures
val averageSalary = TripleItem.Uri("http://data.czso.cz/ontology/prumernaMzda")
val medianSalary = TripleItem.Uri("http://data.czso.cz/ontology/medianMzdy")
val oneOfSalariesMeasures = OneOf(averageSalary,medianSalary)

// czso dimensions
val czsoOntology = "http://data.czso.cz/ontology/"
val czsoRefPeriod = uri(czsoOntology+"refPeriod")
val czsoRefArea = uri(czsoOntology+"refArea")
val czsoSex = uri(czsoOntology+"sex")

val constantsAtObject = RuleConstraint.ConstantsAtPosition.ConstantsPosition.Object
val constantsOnlyAtObject = RuleConstraint.ConstantsAtPosition(constantsAtObject)

val salariesTotal = uri("salaries-total")

uri: String => TripleItem.Uri = ammonite.$sess.cmd9$Helper$$Lambda$2959/0x0000000840dc7040@4ae9c3c3
appliesTo: TripleItem.Uri = LongUri(
  "http://kizi.vse.cz/novp19/diploma-thesis/appliesToRefPeriod"
)
wdProperty: Int => TripleItem.Uri = ammonite.$sess.cmd9$Helper$$Lambda$2960/0x0000000840dc8040@2a771b5a
qbDataSet: TripleItem.Uri = LongUri("http://purl.org/linked-data/cube#dataSet")
averageSalary: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/prumernaMzda"
)
medianSalary: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/medianMzdy"
)
oneOfSalariesMeasures: AtomPattern.AtomItemPattern.OneOf = OneOf(
  ArrayBuffer(
    Constant(LongUri("http://data.czso.cz/ontology/prumernaMzda")),
    Constant(LongUri("http://data.czso.cz/ontology/medianMzdy"))
  )
)
czsoOntology: String = "http://data.czso.cz/ontology/"
czsoRefPeriod: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/refPeriod"
)
czsoRefArea: TripleItem.Uri = LongUri("http://data.czso.cz/ontology/refArea")

# salaries -> alignment

In [11]:
val salariesAlignmentPattern: RulePattern = (
    AtomPattern(subject = 'd', predicate = czsoRefArea, `object` = 'g') &:
    AtomPattern(subject = 'g', predicate = wdProperty(6), `object` = 'f') &:
    AtomPattern(subject = 'f', predicate = wdProperty(6), `object` = 'e') &:   
    AtomPattern(subject = 'e', predicate = wdProperty(102), `object` = 'b') &:
    AtomPattern(subject = 'd', predicate = oneOfSalariesMeasures, `object` = AnyConstant) &:
    AtomPattern(subject = 'd', predicate = qbDataSet, `object` = salariesTotal) &:
    
    AtomPattern(subject = 'd', predicate = czsoRefPeriod, `object` = 'c') &:
    AtomPattern(subject = 'b', predicate = appliesTo, `object` = 'c') &:
    AtomPattern(subject = 'b', predicate = wdProperty(102), `object` = 'a')
    =>:
    AtomPattern(subject = 'a', predicate = wdProperty(1387), `object` = AnyConstant)
)

salariesAlignmentPattern: RulePattern = RulePattern(
  Vector(
    AtomPattern(
      Variable(Variable(3)),
      Constant(LongUri("http://data.czso.cz/ontology/refArea")),
      Variable(Variable(6)),
      Any
    ),
    AtomPattern(
      Variable(Variable(6)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(5)),
      Any
    ),
    AtomPattern(
      Variable(Variable(5)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(4)),
      Any
    ),
    AtomPattern(
      Variable(Variable(4)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      Variable(Variable(1)),
      Any
    ),
    AtomPattern(
      Variable(Variable(3)),
      OneOf(
        ArrayBuffer(
          Constant(LongUri("http://data.czso.cz/ontology/prumernaMzda")),
          Constant(LongUri("http://data.czso.cz/ontology/medianMzdy"))
        )
      ),
      AnyConstant,
      Any
    ),
    AtomPattern(
      Variable(Variable(3

In [12]:
val salariesAlignmentPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(10))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(1))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(salariesAlignmentPattern)

salariesAlignmentPatternTask: com.github.propi.rdfrules.algorithm.RulesMining = com.github.propi.rdfrules.algorithm.amie.Amie@148fb5a0

In [13]:
val startTimeMillis = System.currentTimeMillis()
val salariesAlignmentPatternTaskRuleset = index.mine(salariesAlignmentPatternTask)
println("rules: "+salariesAlignmentPatternTaskRuleset.size)
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

2021-04-20 18:25:54:538 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Predicates trimming.
2021-04-20 18:25:54:627 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Subjects indexing.
2021-04-20 18:25:54:725 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Subjects trimming.
2021-04-20 18:25:54:761 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Objects indexing.
2021-04-20 18:25:54:817 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Objects trimming.
2021-04-20 18:25:54:850 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Amie task settings:
MinHeadSize=1,
MinHeadCoverage=0.0,
MinSupport=1,
MaxThreads=4,
MinAtomSize=0,
MaxRuleLength=10,
WithConstants=true,
ConstantsPosition=Object,
Timeout=60000,
WithDuplicitPredicates=true,
Patterns=List(Mapped(Vector(Mapped(Variable(?d),Constant(Constant(1481837794)),Variable(?g),Any), 

rules: 343
duration: 3s


startTimeMillis: Long = 1618935953967L
salariesAlignmentPatternTaskRuleset: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@516ef029
endTimeMillis: Long = 1618935957482L
durationSeconds: Long = 3L

In [14]:
salariesAlignmentPatternTaskRuleset.export("../rulesets/salariesAlignmentPatternTaskRuleset.txt")

In [15]:
salariesAlignmentPatternTaskRuleset
.filter(rule => rule.measures(Measure.Support).value > 1)
.computePcaConfidence(0.0)
.sortBy(Measure.PcaConfidence, Measure.Support)
.export("../rulesets/salariesAlignmentPatternTaskRulesetFiltered.txt")

# salaries -> party

In [16]:
val salariesPartyPattern: RulePattern = (
    AtomPattern(subject = 'c', predicate = czsoRefArea, `object` = 'f') &:
    AtomPattern(subject = 'f', predicate = wdProperty(6), `object` = 'e') &:
    AtomPattern(subject = 'e', predicate = wdProperty(6), `object` = 'd') &:   
    AtomPattern(subject = 'd', predicate = wdProperty(102), `object` = 'a') &:
    AtomPattern(subject = 'c', predicate = oneOfSalariesMeasures, `object` = AnyConstant) &:
    AtomPattern(subject = 'c', predicate = qbDataSet, `object` = salariesTotal) &:
    AtomPattern(subject = 'c', predicate = czsoRefPeriod, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = appliesTo, `object` = 'b')
    =>:
    AtomPattern(subject = 'a', predicate = wdProperty(102), `object` = AnyConstant)
)

salariesPartyPattern: RulePattern = RulePattern(
  Vector(
    AtomPattern(
      Variable(Variable(2)),
      Constant(LongUri("http://data.czso.cz/ontology/refArea")),
      Variable(Variable(5)),
      Any
    ),
    AtomPattern(
      Variable(Variable(5)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(4)),
      Any
    ),
    AtomPattern(
      Variable(Variable(4)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(3)),
      Any
    ),
    AtomPattern(
      Variable(Variable(3)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      Variable(Variable(0)),
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      OneOf(
        ArrayBuffer(
          Constant(LongUri("http://data.czso.cz/ontology/prumernaMzda")),
          Constant(LongUri("http://data.czso.cz/ontology/medianMzdy"))
        )
      ),
      AnyConstant,
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),


In [17]:
val salariesPartyPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(9))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(1))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(salariesPartyPattern)

salariesPartyPatternTask: com.github.propi.rdfrules.algorithm.RulesMining = com.github.propi.rdfrules.algorithm.amie.Amie@5a7c09f0

In [18]:
val startTimeMillis = System.currentTimeMillis()
val salariesPartyPatternTaskRuleset = index.mine(salariesPartyPatternTask)
println("rules: "+salariesPartyPatternTaskRuleset.size)
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

2021-04-20 18:25:59:216 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Amie task settings:
MinHeadSize=1,
MinHeadCoverage=0.0,
MinSupport=1,
MaxThreads=4,
MinAtomSize=0,
MaxRuleLength=9,
WithConstants=true,
ConstantsPosition=Object,
Timeout=60000,
WithDuplicitPredicates=true,
Patterns=List(Mapped(Vector(Mapped(Variable(?c),Constant(Constant(1481837794)),Variable(?f),Any), Mapped(Variable(?f),Constant(Constant(774364698)),Variable(?e),Any), Mapped(Variable(?e),Constant(Constant(774364698)),Variable(?d),Any), Mapped(Variable(?d),Constant(Constant(52500897)),Variable(?a),Any), Mapped(Variable(?c),OneOf(ArrayBuffer(Constant(Constant(1433642368)), Constant(Constant(1396113393)))),AnyConstant,Any), Mapped(Variable(?c),Constant(Constant(624690160)),Constant(Constant(79965660)),Any), Mapped(Variable(?c),Constant(Constant(-901057765)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(217712947)),Variable(?b),Any)),Some(Mapped(Variable(?a),Constant(Constant(52

rules: 384
duration: 1s


startTimeMillis: Long = 1618935959216L
salariesPartyPatternTaskRuleset: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@290480ec
endTimeMillis: Long = 1618935960225L
durationSeconds: Long = 1L

In [19]:
salariesPartyPatternTaskRuleset.export("../rulesets/salariesPartyPatternTaskRuleset.txt")

In [20]:
salariesPartyPatternTaskRuleset
.filter(rule => rule.measures(Measure.Support).value > 1)
.computePcaConfidence(0.0)
.sortBy(Measure.PcaConfidence, Measure.Support)
.export("../rulesets/salariesPartyPatternTaskRulesetFiltered.txt")

# alignment -> salaries

In [21]:
val alignmentSalariesPattern: RulePattern = (
    AtomPattern(subject = 'a', predicate = czsoRefArea, `object` = 'g') &:
    AtomPattern(subject = 'g', predicate = wdProperty(6), `object` = 'f') &:
    AtomPattern(subject = 'f', predicate = wdProperty(6), `object` = 'e') &:
    AtomPattern(subject = 'e', predicate = wdProperty(102), `object` = 'c') &:  
    AtomPattern(subject = 'd', predicate = wdProperty(1387)) &:
    AtomPattern(subject = 'c', predicate = wdProperty(102), `object` = 'd') &:
    AtomPattern(subject = 'c', predicate = appliesTo, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = czsoRefPeriod, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = qbDataSet, `object` = salariesTotal)
    =>:
    AtomPattern(subject = 'a', predicate = oneOfSalariesMeasures)
)

alignmentSalariesPattern: RulePattern = RulePattern(
  Vector(
    AtomPattern(
      Variable(Variable(0)),
      Constant(LongUri("http://data.czso.cz/ontology/refArea")),
      Variable(Variable(6)),
      Any
    ),
    AtomPattern(
      Variable(Variable(6)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(5)),
      Any
    ),
    AtomPattern(
      Variable(Variable(5)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(4)),
      Any
    ),
    AtomPattern(
      Variable(Variable(4)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      Variable(Variable(2)),
      Any
    ),
    AtomPattern(
      Variable(Variable(3)),
      Constant(LongUri("http://www.wikidata.org/prop/P1387")),
      Any,
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      Variable(Variable(3)),
      Any
    ),
    AtomPattern(
...

In [22]:
val alignmentSalariesPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(10))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(1))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(alignmentSalariesPattern)

alignmentSalariesPatternTask: com.github.propi.rdfrules.algorithm.RulesMining = com.github.propi.rdfrules.algorithm.amie.Amie@4b23cfb1

In [23]:
val startTimeMillis = System.currentTimeMillis()
val alignmentSalariesPatternTaskRuleset = index.mine(alignmentSalariesPatternTask)
println("rules: "+alignmentSalariesPatternTaskRuleset.size)
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

2021-04-20 18:26:01:886 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Amie task settings:
MinHeadSize=1,
MinHeadCoverage=0.0,
MinSupport=1,
MaxThreads=4,
MinAtomSize=0,
MaxRuleLength=10,
WithConstants=true,
ConstantsPosition=Object,
Timeout=60000,
WithDuplicitPredicates=true,
Patterns=List(Mapped(Vector(Mapped(Variable(?a),Constant(Constant(1481837794)),Variable(?g),Any), Mapped(Variable(?g),Constant(Constant(774364698)),Variable(?f),Any), Mapped(Variable(?f),Constant(Constant(774364698)),Variable(?e),Any), Mapped(Variable(?e),Constant(Constant(52500897)),Variable(?c),Any), Mapped(Variable(?d),Constant(Constant(1496784743)),Any,Any), Mapped(Variable(?c),Constant(Constant(52500897)),Variable(?d),Any), Mapped(Variable(?c),Constant(Constant(217712947)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(-901057765)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(624690160)),Constant(Constant(79965660)),Any)),Some(Mapped(Variable(?a),OneOf(Array

rules: 343
duration: 2s


startTimeMillis: Long = 1618935961886L
alignmentSalariesPatternTaskRuleset: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@6403f299
endTimeMillis: Long = 1618935964094L
durationSeconds: Long = 2L

In [24]:
alignmentSalariesPatternTaskRuleset.export("../rulesets/alignmentSalariesPatternTaskRuleset.txt")

In [25]:
alignmentSalariesPatternTaskRuleset
.filter(rule => rule.measures(Measure.Support).value > 1)
.computePcaConfidence(0.0)
.sortBy(Measure.PcaConfidence, Measure.Support)
.export("../rulesets/alignmentSalariesPatternTaskRulesetFiltered.txt")

# party -> salaries 

In [26]:
val partySalariesPattern: RulePattern = (
    AtomPattern(subject = 'a', predicate = czsoRefArea, `object` = 'f') &:
    AtomPattern(subject = 'f', predicate = wdProperty(6), `object` = 'e') &:
    AtomPattern(subject = 'e', predicate = wdProperty(6), `object` = 'd') &:
    AtomPattern(subject = 'd', predicate = wdProperty(102), `object` = 'c') &:  
    AtomPattern(subject = 'c', predicate = wdProperty(102), `object` = AnyConstant) &:
    AtomPattern(subject = 'c', predicate = appliesTo, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = czsoRefPeriod, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = qbDataSet, `object` = salariesTotal)
    =>:
    AtomPattern(subject = 'a', predicate = oneOfSalariesMeasures)
)

partySalariesPattern: RulePattern = RulePattern(
  Vector(
    AtomPattern(
      Variable(Variable(0)),
      Constant(LongUri("http://data.czso.cz/ontology/refArea")),
      Variable(Variable(5)),
      Any
    ),
    AtomPattern(
      Variable(Variable(5)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(4)),
      Any
    ),
    AtomPattern(
      Variable(Variable(4)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(3)),
      Any
    ),
    AtomPattern(
      Variable(Variable(3)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      Variable(Variable(2)),
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      AnyConstant,
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      Constant(
        LongUri("http://kizi.vse.cz/novp19/diploma-thesis/appliesToRefPeriod")
      ),
      Variable(Variable(1)),
   

In [27]:
val partySalariesPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(9))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(1))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(partySalariesPattern)

partySalariesPatternTask: com.github.propi.rdfrules.algorithm.RulesMining = com.github.propi.rdfrules.algorithm.amie.Amie@6eb4f2a0

In [28]:
val startTimeMillis = System.currentTimeMillis()
val partySalariesPatternTaskRuleset = index.mine(partySalariesPatternTask)
println("rules: "+partySalariesPatternTaskRuleset.size)
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

2021-04-20 18:26:05:614 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Amie task settings:
MinHeadSize=1,
MinHeadCoverage=0.0,
MinSupport=1,
MaxThreads=4,
MinAtomSize=0,
MaxRuleLength=9,
WithConstants=true,
ConstantsPosition=Object,
Timeout=60000,
WithDuplicitPredicates=true,
Patterns=List(Mapped(Vector(Mapped(Variable(?a),Constant(Constant(1481837794)),Variable(?f),Any), Mapped(Variable(?f),Constant(Constant(774364698)),Variable(?e),Any), Mapped(Variable(?e),Constant(Constant(774364698)),Variable(?d),Any), Mapped(Variable(?d),Constant(Constant(52500897)),Variable(?c),Any), Mapped(Variable(?c),Constant(Constant(52500897)),AnyConstant,Any), Mapped(Variable(?c),Constant(Constant(217712947)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(-901057765)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(624690160)),Constant(Constant(79965660)),Any)),Some(Mapped(Variable(?a),OneOf(ArrayBuffer(Constant(Constant(1433642368)), Constant(Constant(139611

rules: 384
duration: 1s


startTimeMillis: Long = 1618935965614L
partySalariesPatternTaskRuleset: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@4b3405ff
endTimeMillis: Long = 1618935966898L
durationSeconds: Long = 1L

In [29]:
partySalariesPatternTaskRuleset.export("../rulesets/partySalariesPatternTaskRuleset.txt")

In [30]:
partySalariesPatternTaskRuleset
.filter(rule => rule.measures(Measure.Support).value > 1)
.computePcaConfidence(0.0)
.sortBy(Measure.PcaConfidence, Measure.Support)
.export("../rulesets/partySalariesPatternTaskRulesetFiltered.txt")